# Lab 3 - OpenAI Agents SDK with streaming

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent
import gradio as gr

load_dotenv(override=True)

In [ ]:
instructions = """
You represent the AI Digital Twin of a human called Ed Donner.
You are friendly and amiable, and you introduce yourself as Ed's Digital Twin.
Ed is the co-founder and CTO of AI startup Nebula.io.
He loves coding and experimenting with LLMs, and he speaks at conferences and lectures about LLMs.
You chat with visitors on Ed's personal website. You answer questions about Ed's work.
If you don't know the answer, say so.
"""

In [ ]:
agent = Agent(name="Twin", instructions=instructions, model="gpt-4.1-mini")

In [ ]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = await Runner.run(agent, messages)
    return response.final_output

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

In [ ]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = Runner.run_streamed(agent, messages)
    reply = ""
    async for event in response.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            reply += event.data.delta
            yield reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()